In [1]:
# Necessary Imports
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Enable Eager Execution
tf.enable_eager_execution()

print("Tensorflow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

Tensorflow version: 1.8.0
Eager execution: True


In [3]:
# Load MNIST data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
X=mnist.train.images
Y=mnist.train.labels[:,0].reshape(55000, 1)
print(X.shape)
print(Y.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(55000, 784)
(55000, 1)


In [4]:
# Convert to tensorflow tensors
X = tf.convert_to_tensor(X)
Y = tf.convert_to_tensor(Y)
# Contruct tensorflow dataset object from tensors
tfds = tf.data.Dataset.from_tensors((X,Y))
# View a single example entry from a batch
features, label = iter(tfds).next()
print("example features: ", features[0])
print("example label: ", label[0])

example features:  tf.Tensor(
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0. 

In [5]:
# Simple Logistic Regression Model (Sigmoid)
sig_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(784,))
])

In [6]:
# Define forward propogation
# loss function to be binary crossentropy
def forward_prop(model, x, y):
    a = model(x)
    loss = tf.losses.sigmoid_cross_entropy(y,a)
    return loss

In [7]:
# Define backward propogation
# Object Gradient Tape does the magic of calculating gradients
#tape = tf.GradientTape()
def backward_prop(model, tape, loss):
    grad = tape.gradient(loss, model.variables)
    return grad

In [8]:
# Define optimizer
lr = 1
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [9]:
# The Big For-Loop Training
m = 55000
dc = tf.constant(1/m)
num_epochs = 1000

for epoch in range(num_epochs):
    loss_avg = tfe.metrics.Mean()
    batch_grad = None
    
    for x,y in tfds:
        with tf.GradientTape(persistent=True) as tape:
            loss = forward_prop(sig_model, x, y)
        grad = backward_prop(sig_model, tape, loss)
        loss_avg(loss)
        optimizer.apply_gradients(zip(grad, sig_model.variables))
    
    if epoch % 25 == 24:
        print("Loss: {}".format(loss_avg.result()))

Loss: 0.6950916051864624
Loss: 0.6942474246025085
Loss: 0.6939326524734497
Loss: 0.6937676072120667
Loss: 0.6936651468276978
Loss: 0.6935941576957703
Loss: 0.6935439109802246
Loss: 0.6935044527053833
Loss: 0.6934735774993896
Loss: 0.6934489607810974
Loss: 0.6934274435043335
Loss: 0.6934095025062561
Loss: 0.6933939456939697
Loss: 0.6933801174163818
Loss: 0.6933674216270447
Loss: 0.6933556795120239
Loss: 0.6933448910713196
Loss: 0.6933346390724182
Loss: 0.6933248043060303
Loss: 0.6933144927024841
Loss: 0.6933041214942932
Loss: 0.6932933330535889
Loss: 0.6932802796363831
Loss: 0.6932666897773743
Loss: 0.6932492852210999
Loss: 0.6932272911071777
Loss: 0.693196177482605
Loss: 0.693148136138916
Loss: 0.6930601000785828
Loss: 0.6928481459617615
Loss: 0.6917981505393982
Loss: 0.6783207654953003
Loss: 0.6737080812454224
Loss: 0.6717731356620789
Loss: 0.6704578399658203
Loss: 0.6694847941398621
Loss: 0.6687250137329102
Loss: 0.6681085824966431
Loss: 0.6675958633422852
Loss: 0.6671600937843323


In [10]:
# Test the accuracy model on training and test data

accuracy = tfe.metrics.Accuracy()

for x,y in tfds:
    y_pred = tf.round(sig_model(x))
    y_pred = tf.cast(y_pred, tf.float64)
    accuracy(y_pred, y)

print("Training Accuracy: {}".format(accuracy.result()))

Training Accuracy: 0.9785454545454545


In [11]:
X_t = mnist.test.images
Y_t = mnist.test.labels[:,0]
Y_t = Y_t.reshape(10000, 1)
X_t = tf.convert_to_tensor(X_t)
Y_t = tf.convert_to_tensor(Y_t)
testds = tf.data.Dataset.from_tensors((X_t, Y_t))

accuracy = tfe.metrics.Accuracy()

for x,y in testds:
    y_pred = tf.round(sig_model(x))
    y_pred = tf.cast(y_pred, tf.float64)
    accuracy(y_pred, y)

print("Test Accuracy: {}".format(accuracy.result()))

Test Accuracy: 0.9794
